In [ ]:
import os
from dotenv import load_dotenv
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
load_dotenv()

In [ ]:
# Fetch credentials
DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT')
DB_NAME = os.getenv('DB_NAME')
DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')

# Connect to the database
conn = psycopg2.connect(
    host=DB_HOST,
    port=DB_PORT,
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD
)

In [ ]:
# query = "SELECT obs_time, temp1, temp2 FROM observations;"
cur = conn.cursor()

cur.execute("SELECT * FROM observations;")

# Fetch all results
rows = cur.fetchall()

# Print the results
for row in rows:
    print(row)

# Clean up
cur.close()
conn.close()


In [ ]:
query = "SELECT * FROM observations;"
pd.read_sql_query(query, conn)